In [49]:
%%sh
conda install -y pandas beautifulsoup4

Solving environment: ...working... done

## Package Plan ##

  environment location: /home/is3/miniconda3

  added / updated specs: 
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pytz-2018.4                |           py36_0         212 KB
    pandas-0.23.1              |   py36h637b7d7_0        11.9 MB
    ------------------------------------------------------------
                                           Total:        12.1 MB

The following NEW packages will be INSTALLED:

    pandas: 0.23.1-py36h637b7d7_0
    pytz:   2018.4-py36_0        

Proceed ([y]/n)? 

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


pytz-2018.4          |  212 KB | ########## | 100% 
pandas-0.23.1        | 11.9 MB | ########## | 100% 


In [4]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd

page_link ='https://g.alicdn.com/aliyun/ecs-price-info/2.0.0/price/download/instancePrice.json?spm=0.6883001.instance.1.6250b5sRb5sRx4&file=instancePrice.json'
# fetch the content from url
page_response = requests.get(page_link, timeout=5)
# parse html
page_content = BeautifulSoup(page_response.content, "html5lib")

j = json.loads(soup.text)
j.keys()
# extract all html elements where price is stored
#prices = page_content.find_all(class_='main_price')
# prices has a form:
#[<div class="main_price">Price: $66.68</div>,
# <div class="main_price">Price: $56.68</div>]

# you can also access the main_price class by specifying the tag of the class
#prices = page_content.find_all('div', attrs={'class':'main_price'})

In [127]:
key_structure = j['description'].split("'")[1]
for i in key_structure.split('::'):
    print("d['{}'] = '".format(i))

d['RegionId'] = '
d['InstanceType'] = '
d['NetworkType'] = '
d['OSType'] = '
d['IoOptimized'] = '


In [145]:
l = []

In [100]:
pricing_info = j['pricingInfo']

In [146]:
for k in pricing_info.keys(): #= 'us-east-1::ecs.n4.2xlarge::vpc::linux::optimized'
    d = dict()
    splits = k.split('::')
    d['RegionId'] = splits[0]
    d['InstanceType'] = splits[1]
    d['NetworkType'] = splits[2]
    d['OSType'] = splits[3]
    d['IoOptimized'] = splits[4]
    v = pricing_info[k]

    for key in v.keys():
        for i in v[key]:
            if i['period'] is not '1':
                d[key + '-' + i['period']] = i['price']
            else:
                d[key[:-1]] = i['price']
                
    l.append(d)

In [149]:
df = pd.DataFrame(l)
df.shape

(5372, 13)

In [150]:
df.tail()

,InstanceType,IoOptimized,NetworkType,OSType,RegionId,hour,month,week,year,years-2,years-3,years-4,years-5
5367,ecs.t5-c1m4.2xlarge,optimized,vpc,linux,cn-hangzhou,2.375,684.0,201.6,6480.0,10368.0,11664.0,12441.6,12960.0
5368,ecs.d1.8xlarge,optimized,vpc,linux,cn-hangzhou,24.187,6965.4,2052.96,65988.0,105580.8,118778.4,126696.96,131976.0
5369,ecs.hfg5.xlarge,optimized,vpc,windows,cn-hangzhou,2.385,687.8,202.72,6516.0,10425.6,11728.8,12510.72,13032.0
5370,ecs.gn5-c8g1.4xlarge,optimized,vpc,linux,cn-shanghai,34.409,9908.5,2920.4,93870.0,150192.0,168966.0,180230.4,187740.0
5371,ecs.se1ne.6xlarge,optimized,vpc,linux,us-west-1,12.876,5627.84,1658.73,53316.36,85306.18,95969.45,102367.41,106632.72


In [155]:
set(df.iloc[:,4].tolist())`

{'ap-northeast-1',
 'ap-south-1',
 'ap-southeast-1',
 'ap-southeast-2',
 'ap-southeast-3',
 'ap-southeast-5',
 'cn-beijing',
 'cn-hangzhou',
 'cn-hongkong',
 'cn-huhehaote',
 'cn-qingdao',
 'cn-shanghai',
 'cn-shenzhen',
 'cn-zhangjiakou',
 'eu-central-1',
 'me-east-1',
 'us-east-1',
 'us-west-1'}

In [156]:
import os
os.getcwd()

'/home/is3'

In [157]:
os.mkdir('data')

In [158]:
df.to_csv('data/aliyun.csv')